In [43]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import urllib
import urllib2
from os.path import isfile
from pybloomfilter import BloomFilter

driver = webdriver.Chrome()

In [44]:
keywords = [
                 "suborbital reusable launch vehicle",
                 "suborbital spacecraft vehicle",
                 "suborbital spacecraft spaceplane",
                 "suborbital spacecraft flight",
                 "suborbital spacecraft launch vehicle",
                 "re-entry and hypersonic flight",
                 "high thrust-to-weight",
                 "routine space access",
                 "hypersonic vehicles",
                 "reliable, rapid turnaround aircraft",
                 "Reusable first stage",
                 "Reusable Booster System",
                 "Launch demo payload to orbit",
                 "HTHL",
                 "VTHL",
                 "VTVL",
                 "TSTO",
                 "SSTO",
                 "Ground launch",
                 "Air launch",
                 "barge launch",
                 "Sea launch",
                 "Land downrange",
                 "Return to launch site",
                 "Space access aircraft",
                 "Affordable hypersonic aircraft",
                 "Hypersonic test bed",
                 "operationally responsive spacelift",
                 "ORS",
                 "Modular launch",
                 "Integrated RLV Subsystems",
                 "high propellant mass fraction",
                 "rocketback poststaging maneuver",
                 "autonomously controlled trajectory",
                 "Sequential burning",
                 "Staging Point",
                 "aerodynamic environment",
                 "Return to Launch Site",
                 "Aerodynamic layout",
                 "Aerodynamic configuration",
                 "wing tip mounted vertical tails",
                 "aerodynamic environment",
                 "aerodynamic heating",
                 "aerodynamic force",
                 "aerodynamic characteristics",
                 "CFD",
                 "computational fluid dynamics",
                 "wind tunnel testing",
                 "Propulsion systems",
                 "affordable propulsion systems",
                 "Off-the-Shelf propulsion",
                 "Reusable long life affordable propulsion",
                 "non-toxic propellants",
                 "Management of the propellant",
                 "Pumping systems",
                 "RBCC",
                 "TBCC",
                 "Thermal protection systems",
                 "Durable, low-maintenance thermal protection systems",
                 "TPS and Structures",
                 "Metallic TPS",
                 "Composite TPS",
                 "Hybrid TPS",
                 "Active TPS",
                 "Passive TPS",
                 "hot structure",
                 "Durable thermal structures",
                 "Durable thermal protection",
                 "Composite Hot Structures",
                 "Honeycomb Composites",
                 "Aircraft Hot Wash Structures",
                 "stressing heating",
                 "lightweight structures and components",
                 "Composite structures and tanks",
                 "Robust airframe composition",
                 "composite structures",
                 "Affordable Composite Airframe",
                 "Light weight airframe",
                 "high energy airframe",
                 "Monocoque Tank",
                 "Integral load bearing structure",
                 "Tank Structure Integration",
                 "Autonomous Operations",
                 "Adaptive Guidance and Control",
                 "Integrate with Adaptive G&C",
                 "Autonomous Flight Termination System",
                 "Rangeless range space based command control data acquisition",
                 "safe reliable recovery abort",
                 "Inner-loop Control Allocation",
                 "Adaptive Reconfigurable Control",
                 "Outer-loop Adaptive Guidance",
                 "On-line Trajectory Reshaping",
                 "Integrated Systems Health Management",
                 "More failure tolerance",
                 "Integrated Systems Health Management",
                 "Determine real-time system health",
                 "Integrated System Health Monitoring",
                 "Failure Mode Effects Criticality Analysis",
                 "low-manpower ground operations",
                 "Clean pad minimal infrastructure operations",
                 "aircraft-like operations",
                 "Streamlined “clean pad” operations",
                 "reducing infrastructure and manpower requirements",
                 "enabling flight from a wide range of locations",
                 "lean operations",
                 "Aircraft-Like Ops",
                 "Affordable Infrastructure",
                 "Cycle of Prep Launch Recovery and Turnaround within Single Day",
                 "Rapid Turn Reduces Manpower",
                 "Few Facilities Small Crew Size",
                 "Clean pad - rapid throughput",
                 "Ops Control Center – like aircraft",
                 "Containerized payloads",
                 "Standard interfaces processes",
                 "Low man-power aircraft-like operations"
                 ]
class PaperInfo():
    def __init__(self):
        self.title = ''
        self.citedTimes = 0
        self.fromUrl = ''
        self.referenceList = []
        self.abstract = ''
        self.conference= ''
        self.author= ''
        self.publish_time = ''
        self.current_time = time.strftime('%Y-%m-%d %H:%M:%S')
    def reset(self):
        __init__(self)
    def __str__(self):
        shortAbstract = self.abstract[0:30]+' ... '+self.abstract[-31:-1] if len(self.abstract)>60 else self.abstract
        return '%-15s: %s\n%-15s: %s\n%-15s: %s\n%-15s: %s\n%-15s: %s\n%-15s: %s\n%-15s: %s\n%-15s: %s\n%-15s: %s' % \
            ('title',self.title, 'citedTimes', self.citedTimes,'fromUrl', self.fromUrl,\
             'referenceList', self.referenceList, 'abstract', shortAbstract,\
             'conference', self.conference, 'author', self.author, \
             'publish_time', self.publish_time, 'current_time', self.current_time)
    def saveToMysql(self):
        print 'save to mysql, not implement yet'

#从形如“Times Cited: 1”中解析出被引用次数
def parseReferenceCount(reference):
    if 'Times Cited: ' not in reference:
        print 'not sci reference'
        return 0
    pos1 = reference.find(': ')
    pos2 = reference.find('(')
    if pos1 == -1:
        return 0
    else:
        return int(reference[pos1+1:pos2])
#选择english,方便解析
def chooseEnglishLanguage():
    languageElement = driver.find_element_by_xpath('//div[1]/div[12]/ul[2]/li[3]/a')
    if languageElement.get_attribute('title') == 'English': return
    languageElement.click()
    englishItem = driver.find_element_by_xpath('//div[1]/div[12]/ul[2]/li[3]/ul/li[3]/a')
    assert englishItem.get_attribute('title') == 'English' , 'ERROR: not english language'
    englishItem.click()
    
def parseReference():
    resultCount = int(driver.find_element_by_id('hitCount.top').text)
    pageCount = int(driver.find_element_by_id('pageCount.top').text)
    count = 1;
    titleList = []
    for pageNum in range(1,pageCount+1):
        items = driver.find_elements_by_class_name('search-results-item')
        for item in items:
            reference_title = item.find_element_by_class_name('reference-title').text
            titleList.append(reference_title)
            count += 1
            if count>resultCount : break
        #下一页
        nextPage = driver.find_element_by_class_name('paginationNext')
        if nextPage.get_attribute('href') == 'javascript: void(0)':
            break
        nextPage.click()
    for pageNum in range(pageCount-1):
        driver.back()
    return titleList

In [45]:
filterPath = '/Users/shibo/filter.bloom'
bf = BloomFilter.open(filterPath) if isfile(filterPath) else BloomFilter(1000000, 0.001, filterPath)

driver.get('http://apps.webofknowledge.com/')
time.sleep(5)
chooseEnglishLanguage()
print driver.title

form = driver.find_element_by_name("value(input1)")
form.clear()
form.send_keys("suborbital reusable launch vehicle aircraft flight experiment")
form.submit()
print driver.title

resultCount = int(driver.find_element_by_id('hitCount.top').text)
print 'records in total: ', resultCount
pageCount = int(driver.find_element_by_id('pageCount.top').text)
print 'total page number: ' , pageCount
count = 0;
#将paper信息保存在paperInfo对象中
paperInfo = PaperInfo()
#一页一页下载
for pageNum in range(1,pageCount+1):
    #爬取本页内部链接
    for i in range(1 , 11):
        count = count +1
        if count > resultCount:
            break
        xpath = '//*[@id="RECORD_'+ str(count) +'"]/div[3]/div[1]/div/a'
        paperLink = WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,xpath)))
        #field
        title = paperLink.text  
        #检查是否已经下载过
        if title in bf:continue
        else: bf.add(title)
        paperInfo.title = title
        citedTimes = driver.find_element_by_xpath('//*[@id="RECORD_'+ str(count) +'"]/div[4]/div').text
        #field
        paperInfo.citedTimes = parseReferenceCount(citedTimes)
        paperLink.click()
        #field
        paperInfo.fromUrl = driver.current_url#此url可以获取，但无法直接访问，只能从结果列表中点击，因为sci的限制
        #field
        referenceList = []
        try:
            referenceXpath = '//*[@id="records_form"]/div/div/div/div[2]/div[2]/div/div/p[2]/a'
            reference = driver.find_element_by_xpath(referenceXpath)
            reference.click()
            referenceList = parseReference()
            driver.back()
        except NoSuchElementException, e:
            print 'no reference'
        paperInfo.referenceList = referenceList    
        
        blocks  = driver.find_elements_by_class_name('block-record-info')
        for block in blocks:
            try:
                title3 = block.find_element_by_class_name('title3').text
                if title3 == 'Abstract':
                    #field
                    paperInfo.abstract = block.find_element_by_class_name('FR_field').text
                elif title3 == 'Conference':
                    fr_fields = block.find_elements_by_class_name('FR_field')
                    for fr_field in fr_fields:
                        fr_label = fr_field.find_element_by_class_name('FR_label').text
                        if fr_label == 'Conference:':
                            #field
                            paperInfo.conference = fr_field.find_element_by_tag_name('value').text
                            break
            except NoSuchElementException , e:
                fr_fields = block.find_elements_by_class_name('FR_field')
                for fr_field in fr_fields:
                    fr_label = fr_field.find_element_by_class_name('FR_label').text
                    if fr_label == 'By:':
                        #field
                        paperInfo.author = fr_field.text[4:]
                    elif fr_label == 'Published:':
                        #field
                        paperInfo.publish_time = fr_field.text[len('Published:')+1 : ]
        #field
        paperInfo.current_time = time.strftime('%Y-%m-%d %H:%M:%S')
        paperInfo.saveToMysql()
        print paperInfo
        paperInfo.reset()
        driver.back()
        
        print '================'
    #下一页
    nextPage = driver.find_element_by_class_name('paginationNext')
    if nextPage.get_attribute('href') == 'javascript: void(0)':
        break
    nextPage.click()

Web of Science [v.5.18] - All Databases Home
Web of Science [v.5.18] - All Databases Results
records in total:  4
total page number:  1


In [47]:
for index , keyword in enumerate(keywords):
    print str(index) +':' + keyword

0:suborbital reusable launch vehicle
1:suborbital spacecraft vehicle
2:suborbital spacecraft spaceplane
3:suborbital spacecraft flight
4:suborbital spacecraft launch vehicle
5:re-entry and hypersonic flight
6:high thrust-to-weight
7:routine space access
8:hypersonic vehicles
9:reliable, rapid turnaround aircraft
10:Reusable first stage
11:Reusable Booster System
12:Launch demo payload to orbit
13:HTHL
14:VTHL
15:VTVL
16:TSTO
17:SSTO
18:Ground launch
19:Air launch
20:barge launch
21:Sea launch
22:Land downrange
23:Return to launch site
24:Space access aircraft
25:Affordable hypersonic aircraft
26:Hypersonic test bed
27:operationally responsive spacelift
28:ORS
29:Modular launch
30:Integrated RLV Subsystems
31:high propellant mass fraction
32:rocketback poststaging maneuver
33:autonomously controlled trajectory
34:Sequential burning
35:Staging Point
36:aerodynamic environment
37:Return to Launch Site
38:Aerodynamic layout
39:Aerodynamic configuration
40:wing tip mounted vertical tails
41